In [ ]:
%load_ext autoreload
%autoreload 2

from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
import geopandas as gpd
import pandas as pd
import json
import folium
from folium import Map, Polygon
from shapely import intersection_all
from shapely.geometry import mapping
import shapely

import numpy as np

## Search

In [ ]:
file_aoi = "/data/aoi_bretagne.geojson"
gj_aoi = read_geojson(file_aoi)
aoi = geojson_to_wkt(gj_aoi["features"][0])
with open("/data/creds.json") as f:
    creds = json.load(f)
api = SentinelAPI(creds["username"], creds["password"])
products = api.query(
    aoi,
    producttype="SLC",
    date=("20220110", "20220430"),
    #  orbitdirection='ASCENDING',
    limit=50,
)
gdf = api.to_geodataframe(products).reset_index(names='id')

## group and display

In [ ]:
map_ = Map()
grouped = gdf.groupby(["relativeorbitnumber", "slicenumber"])
groups = list(grouped.groups.keys())
ll = gdf.total_bounds.reshape(2, 2)
bbox = list(map(lambda x: [x[1], x[0]], ll))
for g in groups:
    # convert Multipolygons to Polygons
    dfg = grouped.get_group(g).explode(index_parts=False).to_crs("EPSG:4236")

    # find the intersection of all geometries
    inter = intersection_all(dfg.geometry)

    # a collection is required for folium styling props
    gj = {}
    gj["type"] = "FeatureCollection"
    gj["features"] = []
    item = mapping(inter)
    item["properties"] = {}
    if all(dfg.orbitdirection == "ASCENDING"):
        item["properties"]["color"] = "cadetblue"
    elif all(dfg.orbitdirection == "DESCENDING"):
        item["properties"]["color"] = "orange"
    else:
        # not sure if this may happen
        item["properties"]["color"] = "red"
    gj["features"].append(item)
    folium_products = folium.GeoJson(
        data=gj,
        style_function=lambda x: {
            "fillColor": "lightblue",
            "color": x["properties"]["color"],
        }
    )
    folium_aoi = folium.GeoJson(
        data=gj_aoi,
        style_function=lambda x: {
            "fillColor": "none",
            "color": "grey",
        }
    )
    date_ts = dfg["beginposition"].dt.strftime("%Y-%m-%d %X")
    date_str = "<br>".join(date_ts.index.astype(str).str.cat(date_ts.values, sep=": "))
    folium.Tooltip(f"index: date<br>{date_str}").add_to(folium_products)
    folium.Tooltip(f"Area of Interest").add_to(folium_aoi)
    folium_products.add_to(map_)
    folium_aoi.add_to(map_)
    # print(inter)
map_.fit_bounds(bbox)
map_

## download

In [ ]:
# fill indices of products to download
indices_to_download = [8, 13]
ids = gdf['id'][indices_to_download]
dl = api.download_all_quicklooks(ids, directory_path='/data/S1/')

In [ ]:
# extract paths for processing
file_paths = [item[1]['path'] for item in dl[0].items()]